In [72]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 500)

import seaborn as sns
import nltk
nltk.download('words')
pd.set_option('max_colwidth', 40)
import os
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
sws = set(stopwords.words('english'))
punctuation = set(string.punctuation)
                            

[nltk_data] Downloading package words to /home/ubuntu/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [73]:
df = pd.read_json('data/data.json')

In [74]:
df_text = df[['name','description']]

In [79]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)
import re
from string import digits
import nltk
from html.parser import HTMLParser

'''you must have nltk packages installed to run this file. To do so run the following commands:

import ntlk 
nltk.download()

A new window should open, showing the NLTK Downloader. Click on the File menu and select Change Download Directory. 
For central installation, set this to C:\nltk_data (Windows), /usr/local/share/nltk_data (Mac)'''

class MLStripper(HTMLParser):
    '''This class parses HTML from the description column
    input: string
    output: string'''
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

def clean_desc(df):

    '''cleans the data. see below for documentation. adds three new columns:
    df['description_clean'] is the new cleaned description data
    df['description_html'] are the descriptions written in html
    df['description_none'] are the descriptions with no text
    '''
    
    #creates a new column for all events using HTML invites
    df['description_html'] = df['description'].apply(lambda x: True if ('</span>' or '</div>') in x else False)

    #strips HTML documentation
    df['description_clean'] = df['description'].apply(lambda x: strip_tags(x))

    #replaces artifacts from HTML tht didn't get cleaned up through the strip_tags script
    df['description_clean'] = df['description_clean'].apply(lambda x: x.replace('\r',''))
    df['description_clean'] = df['description_clean'].apply(lambda x: x.replace('\n',''))
    df['description_clean'] = df['description_clean'].apply(lambda x: x.replace('</li>',''))
    df['description_clean'] = df['description_clean'].apply(lambda x: x.replace('<li>',' '))
    
    #remove websites
    df['description_clean'] = df['description_clean'].apply(lambda x: re.sub(r'www\.\S+\.com', '', x, flags=re.MULTILINE))

    #remove websites
    df['description_clean'] = df['description_clean'].apply(lambda x: re.sub(r'^https?:\/\/.*[\r\n]*', '', x, flags=re.MULTILINE))
    
    #remove email addresses
    df['description_clean'] = df['description_clean'].apply(lambda x: re.sub('\S*@\S*\s?','', x))

    #remove special characters
    df['description_clean'] = df['description_clean'].apply(lambda x: re.sub('\W+',' ', x )) 
    
    # remove numbers from string
    df['description_clean'] = df['description_clean'].apply(lambda x: re.sub(r'\d+', '', x))
    
    # lower case everything
    df['description_clean'] = df['description_clean'].apply(lambda x: x.lower())

    #removes non-english words
    words = set(nltk.corpus.words.words())
    df['description_clean'] = df['description_clean'].apply(lambda x: " ".join(w for w in nltk.wordpunct_tokenize(x) if w.lower() in words or not w.isalpha())) 

    df['description_none']= df['description_clean']==''

    return df

In [80]:
def description_cols():
    df1 = clean_desc(df_description['description'].apply(strip_tags))
    return df1

In [18]:
df_description['description'] = df_description['description'].apply(strip_tags)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [19]:
description_df = clean_desc(df_description)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/sta

In [27]:
description_df['description_clean'] = \
            [''.join([word for word in doc if word not in punctuation]) for doc in description_df['description_clean']]

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [29]:
tfidf1 = TfidfVectorizer(stop_words=sws,max_features=20000)
description_vector = tfidf1.fit_transform(description_df['description_clean'])

description_word_df = pd.DataFrame(description_vector.toarray(),columns = tfidf1.get_feature_names())

In [30]:
tfidf2 = TfidfVectorizer(stop_words=sws,max_features=10000)
name_vector = tfidf2.fit_transform(df['name'])
name_word_df = pd.DataFrame(name_vector.toarray(),columns = tfidf2.get_feature_names())

In [35]:
print((description_word_df + name_word_df).shape)
print(description_word_df.shape)
print(name_word_df.shape)

(14337, 25465)
(14337, 20000)
(14337, 10000)


In [40]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=400)
description_topics = svd.fit_transform(description_word_df)
pd.DataFrame(description_topics,columns

In [45]:
topic_cols = ['topic_{}'.format(i) for i in range(description_topics.shape[1])]
description_topics = pd.DataFrame(description_topics,columns=topic_cols)

In [47]:
description_topics.to_pickle('data/description_topics_df.pickle')

In [454]:
#description_word_df.to_pickle('data/description_df.pickle')
description_word_df.to_pickle('data/description_word_df.pickle')

In [ ]:
name_word_df